In [24]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import wrapcauchy

def correlated_random_walk_cauchy(n_steps, step_size=1, cauchy_coeff=0.5):
    """
    Genera una trayectoria de CRW con ángulos distribuidos según la distribución de Cauchy.
    
    Parameters:
    - n_steps: Número de pasos
    - step_size: Tamaño de los pasos
    - cauchy_coeff: Coeficiente de Cauchy para controlar el comportamiento de los ángulos
    
    Returns:
    - df: DataFrame con las posiciones x, y
    """
    # Generar ángulos con distribución de Cauchy
    angles = wrapcauchy.rvs(cauchy_coeff, size=n_steps)
    positions = [[0, 0]]  # Iniciar en el origen
    
    for i in range(1, n_steps):
        dx = step_size * np.cos(angles[i])
        dy = step_size * np.sin(angles[i])
        new_position = [positions[-1][0] + dx, positions[-1][1] + dy]
        positions.append(new_position)
    
    df = pd.DataFrame(positions, columns=['x_pos', 'y_pos'])
    return df, angles
def turning_angles(df):
    """
    Calcula los ángulos de giro para una trayectoria dada.
    
    Parameters:
    df (pandas.DataFrame): DataFrame con las posiciones ['x_pos', 'y_pos'].
    
    Returns:
    numpy.array: Ángulos de giro entre cada tres puntos consecutivos.
    """
    angles = []
    for i in range(1, len(df) - 1):
        # Vectores entre puntos consecutivos
        vec1 = df.iloc[i] - df.iloc[i-1]
        vec2 = df.iloc[i+1] - df.iloc[i]
        
        # Producto punto y norma
        dot_product = np.dot(vec1, vec2)
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        
        # Calcular el ángulo
        angle = np.arccos(dot_product / (norm1 * norm2))
        angles.append(angle)
    
    return np.array(angles)
# Generar dos trayectorias CRW con diferentes coeficientes de Cauchy
crw_trajectory_1, source_angles_1 = correlated_random_walk_cauchy(n_steps=100, cauchy_coeff=0.5)
crw_trajectory_2, source_angles_2 = correlated_random_walk_cauchy(n_steps=100, cauchy_coeff=0.8)

# Calcular los ángulos de giro observados para ambas trayectorias
observed_angles_1 = turning_angles(crw_trajectory_1)
observed_angles_2 = turning_angles(crw_trajectory_2)

# Graficar las distribuciones
fig = go.Figure()

# Histograma de ángulos observados para la primera trayectoria
fig.add_trace(go.Histogram(x=observed_angles_1, nbinsx=30, histnorm='probability', name='Observed Angles 1', opacity=0.7))

# Histograma de ángulos observados para la segunda trayectoria
fig.add_trace(go.Histogram(x=observed_angles_2, nbinsx=30, histnorm='probability', name='Observed Angles 2', opacity=0.7))

# Curva de distribución de Cauchy para la primera trayectoria
x_vals = np.linspace(-np.pi, np.pi, 100)
y_vals_1 = wrapcauchy.pdf(x_vals, 0.5)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_1, mode='lines', name='Source Distribution 1'))

# Curva de distribución de Cauchy para la segunda trayectoria
y_vals_2 = wrapcauchy.pdf(x_vals, 0.8)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_2, mode='lines', name='Source Distribution 2'))

# Configuración del gráfico
fig.update_layout(title='Turning-Angle Distribution: Observed vs Source',
                  xaxis_title='Angle (radians)',
                  yaxis_title='Probability',
                  barmode='overlay')

fig.show()
